<a href="https://colab.research.google.com/github/Eliandry/Neural-networks/blob/main/guessEmotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from keras.models import Sequential
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, GlobalAveragePooling2D,MaxPooling2D,BatchNormalization
from google.colab import files
from PIL import Image
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.losses import categorical_crossentropy
from keras.applications import vgg19
from sklearn.utils import shuffle
import cv2
import pandas as pd
import warnings
from keras.initializers import  RandomNormal
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard

In [17]:
warnings.filterwarnings("ignore")

data = pd.read_csv('./fer2013.csv')
width, height = 48, 48
datapoints = data['pixels'].tolist()

    # получаем все свойства для дальнейшего обучения
X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

    # получем лэйблы для обучения
y = pd.get_dummies(data['emotion']).values

In [18]:
X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.7, random_state=41)

In [20]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 20
width, height = 48, 48

In [21]:
model = Sequential()


model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', input_shape=(48,48,1), bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', input_shape=(48,48,1), bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.25))

# 3rd 
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.25))
          
# 5th 
model.add(Conv2D(128, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.25))

# 7th 
model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same', bias_initializer=RandomNormal(stddev=1), kernel_initializer=RandomNormal(stddev=1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
model.add(Dropout(0.1))


model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(7, activation='softmax'))

In [22]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [24]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=64,
          epochs=30,
          verbose=1,
          validation_data=(np.array(X_val), np.array(y_val)),
          shuffle=True)

Epoch 1/30
51/51 [==============================] - 152s 3s/step - loss: 2.8575 - accuracy: 0.2103 - val_loss: 2.1753 - val_accuracy: 0.1957
Epoch 2/30
51/51 [==============================] - 150s 3s/step - loss: 1.8141 - accuracy: 0.2577 - val_loss: 1.8218 - val_accuracy: 0.2180
Epoch 3/30
51/51 [==============================] - 150s 3s/step - loss: 1.7856 - accuracy: 0.2598 - val_loss: 1.7991 - val_accuracy: 0.2586
Epoch 4/30
51/51 [==============================] - 149s 3s/step - loss: 1.7800 - accuracy: 0.2601 - val_loss: 1.8273 - val_accuracy: 0.2619
Epoch 5/30
51/51 [==============================] - 149s 3s/step - loss: 1.7566 - accuracy: 0.2614 - val_loss: 1.9258 - val_accuracy: 0.2624
Epoch 6/30
51/51 [==============================] - 150s 3s/step - loss: 1.7541 - accuracy: 0.2735 - val_loss: 1.9948 - val_accuracy: 0.2693
Epoch 7/30
51/51 [==============================] - 150s 3s/step - loss: 1.7449 - accuracy: 0.2790 - val_loss: 2.0329 - val_accuracy: 0.2562
Epoch 8/30
51

In [28]:
imgs = Image.open(f'/content/2.jpg').convert('L')
imgs=np.array(imgs)
imgs=vgg19.preprocess_input(np.expand_dims(imgs,axis=0))
imgs.shape=(imgs.shape[0],48,48,1)
print(imgs.shape)
model.predict(imgs)

(1, 48, 48, 1)


array([[9.9999464e-01, 7.5959491e-26, 4.7445587e-06, 6.0346304e-07,
        7.9840167e-28, 3.0806468e-30, 5.0739136e-17]], dtype=float32)